# `RankingPipeline`

Dans ce script on teste la pipeline complète.

## Match scores

In [1]:
import os
from getpass import getpass

hf_token = getpass()
os.environ["HF_HUB_CACHE"] = "/home/jupyterhub-users/shared/projet_comparia/huggingface_hub/"
os.environ["HF_DATASETS_CACHE"] = "/home/jupyterhub-users/shared/projet_comparia/huggingface_hub/"
os.environ["HF_TOKEN"] = hf_token

In [2]:
from rank_comparia.pipeline import RankingPipeline

In [3]:
pipeline = RankingPipeline(
    method="elo_random",
    include_votes=True,
    include_reactions=True,
    bootstrap_samples=5,
    batch=False,
    export_graphs=False,
)

Final votes dataset contains 55617 conversations pairs.
Reactions data originally contains 22993 conversations pairs.
Final reactions dataset contains 21244 conversations pairs.


In [ ]:
pipeline.matches

In [ ]:
pipeline.match_list()

In [ ]:
pipeline.run()

## Alternative pipeline

Needs to be fixed.

In [ ]:
pipeline = RankingPipeline(
    method="ml", include_votes=True, include_reactions=True, bootstrap_samples=100, batch=False, export_graphs=False
)

In [ ]:
pipeline.run()

## Ranking by category

In [4]:
pipeline = RankingPipeline(
    method="elo_random",
    include_votes=True,
    include_reactions=True,
    bootstrap_samples=5,
    batch=False,
    export_graphs=False,
)

Final votes dataset contains 55617 conversations pairs.
Reactions data originally contains 22993 conversations pairs.
Final reactions dataset contains 21244 conversations pairs.


In [5]:
pipeline.run_category("Education")

Computing bootstrap scores from a sample of 23033 matches.


Processing bootstrap samples: 100%|██████████| 5/5 [00:00<00:00, 25.83it/s]


model,median,p2.5,p97.5
str,f64,f64,f64
"""gemini-2.0-flash-exp""",1182.065382,1123.96172,1197.349377
"""claude-3-7-sonnet""",1142.79951,1105.751388,1173.081962
"""gemma-3-27b""",1140.554535,1044.594219,1148.013817
"""gemini-2.0-flash-001""",1114.657042,1056.640179,1154.067566
"""command-a""",1110.365571,1053.626364,1114.628192
…,…,…,…
"""phi-3.5-mini-instruct""",892.843738,880.152552,930.653694
"""mixtral-8x22b-instruct-v0.1""",873.42931,841.922332,900.226367
"""chocolatine-2-14b-instruct-v2.…",821.695193,770.66251,919.042046


In [6]:
results = pipeline.run_all_categories()

Computing bootstrap scores from a sample of 23033 matches.


Processing bootstrap samples: 100%|██████████| 5/5 [00:00<00:00, 24.85it/s]


Computing bootstrap scores from a sample of 8046 matches.


Processing bootstrap samples: 100%|██████████| 5/5 [00:00<00:00, 74.57it/s]


Computing bootstrap scores from a sample of 12297 matches.


...

Processing bootstrap samples: 100%|██████████| 5/5 [00:00<00:00, 28301.65it/s]


Computing bootstrap scores from a sample of 0 matches.


Processing bootstrap samples: 100%|██████████| 5/5 [00:00<00:00, 33662.15it/s]


Computing bootstrap scores from a sample of 1 matches.


Processing bootstrap samples: 100%|██████████| 5/5 [00:00<00:00, 32164.91it/s]
